In [ ]:
from sandbox.qemu import QemuVMManager, VMConfig, SSHConfig
from pathlib import Path

In [3]:
Path("./docker/shared").resolve().as_posix()
Path("/shared").name
(Path("./docker/shared") / "TestVM").name

'TestVM'

In [ ]:
import time


cfg = VMConfig(
    container_name="TestVM",
    vm_base_dir="./docker/vms/ubuntu-noble",
    host_shared_dir="./docker/shared",
    server_host_dir="./server",
    extra_ports={8000: 8000},
    runtime_env={
        "HOST": "0.0.0.0",
        "PORT": "8000",
        "DISPLAY": ":0",
    },
)


ssh_cfg = SSHConfig(username="user", password="password", port=2222)
vm = QemuVMManager(config=cfg, ssh_config=ssh_cfg)

# This will create (or reuse) the container, start it, wait for SSH and deploy your server:
vm.get_or_create_container(start_if_stopped=True, setup_if_needed=False)

# 2) Run a one‑off command over SSH
with vm.get_ssh_client() as ssh:
    res = ssh.exec_command("uname -a")
    print("Kernel:", res["stdout"].strip())

    # 3) Now drop into a fully interactive shell
    ssh.open_shell()
# when you exit that shell, the SSH connection is automatically cleaned up


2025-04-17 17:36:25,328 - QemuVMManager - INFO - VM state: initializing -> creating


2025-04-17 17:36:50,612 - QemuVMManager - INFO - Preparing container configuration for TestVM
2025-04-17 17:36:50,634 - QemuVMManager - INFO - Container TestVM already exists
2025-04-17 17:36:50,635 - QemuVMManager - INFO - Reusing existing container TestVM
2025-04-17 17:36:50,635 - QemuVMManager - INFO - VM state: creating -> creating
2025-04-17 17:37:15,790 - QemuVMManager - INFO - Preparing container configuration for TestVM
2025-04-17 17:37:15,807 - QemuVMManager - INFO - Container TestVM already exists
2025-04-17 17:37:15,808 - QemuVMManager - INFO - Reusing existing container TestVM
2025-04-17 17:37:15,808 - QemuVMManager - INFO - VM state: creating -> starting
2025-04-17 17:37:16,133 - QemuVMManager - INFO - Container started, waiting for VM to boot...
2025-04-17 17:37:16,133 - QemuVMManager - INFO - Waiting for SSH connection to be available (timeout: 180s)
2025-04-17 17:37:16,134 - QemuVMManager - INFO - Executing SSH command: echo 'SSH connection test'
2025-04-17 17:37:30,249

VMOperationError: Guest setup failed: Server startup failed with codes [0, 1]

In [ ]:
# manager.get_or_create_container(setup_if_needed=True)
# manager.get_or_create_container()
time.sleep(240)
manager.stop_and_cleanup()

: 